In [26]:
import os, sys
from pathlib import Path
import numpy as np
import math
import psutil

In [39]:
def get_size_MB(shape):
    current_size = math.prod(shape)/1024**3
    current_size *= 1
    current_size *= 1024
    return round(current_size,4)

def get_size_GB(shape, dtype=None):
    current_size = math.prod(shape)/1024**3
    current_size *=2
    
    return current_size

def optimize_chunk_shape_3d(image_shape, origional_chunks, output_chunks=(1,1024,1024)):
    
    dtype='uint16'
    cpu_cores=os.cpu_count()
    sim_jobs=4
    mem=int((psutil.virtual_memory().free/1024**3)*.8)
    chunk_limit_GB = mem / cpu_cores / 8
    y = origional_chunks[1] if origional_chunks[1] > output_chunks[1] else output_chunks[1]
    x = origional_chunks[2] if origional_chunks[2] > output_chunks[2] else output_chunks[2]

    origional_chunks = (origional_chunks[0], y, x)

    current_chunks = origional_chunks
    current_size = get_size_GB(current_chunks, dtype)

    print(current_chunks)
    print(current_size)

    if current_size > chunk_limit_GB:
        return current_chunks

    idx = 0
    chunk_bigger_than_z = True if current_chunks[0] >= image_shape[0] else False
    chunk_bigger_than_y = True if current_chunks[1] >= image_shape[1] else False
    chunk_bigger_than_x = True if current_chunks[2] >= image_shape[2] else False

    while current_size <= chunk_limit_GB:

        # last_size = get_size_GB(current_chunks,dtype)
        last_shape = current_chunks

        # chunk_iter_idx = idx % 2
        # if chunk_iter_idx == 0 and chunk_bigger_than_y == False:
        #     current_chunks = (origional_chunks[0], current_chunks[1] + output_chunks[1], current_chunks[2])
        # elif chunk_iter_idx == 1 and chunk_bigger_than_x == False:
        #     current_chunks = (origional_chunks[0], current_chunks[1], current_chunks[2] + output_chunks[2])

        # Iterate over y first then x
        if chunk_bigger_than_y == False:
            current_chunks = (origional_chunks[0],current_chunks[1]+output_chunks[1],current_chunks[2])
        elif chunk_bigger_than_x == False:
            current_chunks = (origional_chunks[0],current_chunks[1],current_chunks[2]+output_chunks[2])
        elif chunk_bigger_than_z == False:
            current_chunks = (origional_chunks[0] + output_chunks[0], current_chunks[1], current_chunks[2])

        current_size = get_size_GB(current_chunks, dtype)

        print(current_chunks)
        print(current_size)
        print('next step chunk limit {}'.format(current_size))

        if current_size > chunk_limit_GB:
            return last_shape

        if current_chunks[0] > image_shape[0]:
            chunk_bigger_than_z = True

        if current_chunks[1] > image_shape[1]:
            chunk_bigger_than_y = True

        if current_chunks[2] > image_shape[2]:
            chunk_bigger_than_x = True

        if all([chunk_bigger_than_z, chunk_bigger_than_y, chunk_bigger_than_x]):
            return last_shape

        idx += 1

    


In [24]:
shape = (138, 854, 568)
print(f'{shape} size in MB={get_size_MB(shape)}')
shape = (1, 36962//8, 43442//8)
print(f'{shape} size in MB={get_size_MB(shape)}')

(138, 854, 568) size in MB=63.8389
(1, 4620, 5430) size in MB=23.9244


In [40]:
#optimum_chunks = 
#utils.optimize_chunk_shape_3d_2(
#test_image.shape,
#test_image.chunks,
#self.origionalChunkSize[2:],test_image.dtype,self.res0_chunk_limit_GB)
   

image_shape = (4750, 36962, 43442)
original_chunks = (1, 1024, 1024)
image_chunks=(250, 1536, 1024)
chunks = optimize_chunk_shape_3d(image_shape, image_chunks)
print(chunks)

(250, 1536, 1024)
0.732421875
(250, 1536, 1024)
